In [ ]:
! pip install pymysql python-dotenv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

print("Environment variables loaded!")

## 1. Sản xuất cà phê trong nước

In [ ]:
df_production = pd.DataFrame(production)
df_production

In [ ]:
df_production.info()

In [ ]:
# Đảm bảo các cột số thực sự ở dạng float
numeric_cols = ['area_thousand_ha', 'output_tons', 'export_tons']
df_production[numeric_cols] = df_production[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Thực hiện nội suy đầy đủ theo hai chiều
df_production = df_production.interpolate(method='linear', limit_direction='both')

# Kiểm tra lại kết quả
print(df_production.info())
print(df_production)
print("\nSố lượng giá trị null còn lại:")
print(df_production.isnull().sum())

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
sns.scatterplot(data=df_production, x='year', y='area_thousand_ha')
plt.title('Area (thousand ha) over Time')

plt.subplot(1, 3, 2)
sns.scatterplot(data=df_production, x='year', y='output_tons')
plt.title('Output (tons) over Time')

plt.subplot(1, 3, 3)
sns.scatterplot(data=df_production, x='year', y='export_tons')
plt.title('Export (tons) over Time')

plt.tight_layout()
plt.show()


In [ ]:
cursor.execute("select * from weather")
weather = cursor.fetchall()
df_weather = pd.DataFrame(weather)

In [ ]:
df_weather

## Part 2: Kim Ngạch Xuất Khẩu và Giá Cà Phê

#Phân tích mối quan hệ giữa kim ngạch xuất khẩu và giá cà phê thế giới/Việt Nam


In [ ]:
# Fetch coffee export data (kim ngạch + giá)
cursor.execute("SELECT * FROM coffee_export")
coffee_export = cursor.fetchall()
df_export = pd.DataFrame(coffee_export)

print("Coffee Export Data:")
print(df_export.head())
print(f"\nShape: {df_export.shape}")
print(f"\nColumns: {df_export.columns.tolist()}")

In [ ]:
# Scatter Plot: Kim Ngạch vs Giá Cà Phê
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Kim ngạch vs Giá thế giới
axes[0].scatter(df_export['price_world_usd_per_ton'], df_export['export_value_million_usd'], 
                s=100, alpha=0.6, c=df_export['year'], cmap='viridis', edgecolors='black')
axes[0].set_xlabel('Giá Cà Phê Thế Giới (USD/tấn)', fontsize=12)
axes[0].set_ylabel('Kim Ngạch Xuất Khẩu (Triệu USD)', fontsize=12)
axes[0].set_title('Kim Ngạch XK vs Giá Thế Giới', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
cbar1 = plt.colorbar(axes[0].collections[0], ax=axes[0])
cbar1.set_label('Năm', fontsize=10)

# Kim ngạch vs Giá Việt Nam
axes[1].scatter(df_export['price_vn_usd_per_ton'], df_export['export_value_million_usd'], 
                s=100, alpha=0.6, c=df_export['year'], cmap='plasma', edgecolors='black')
axes[1].set_xlabel('Giá Cà Phê Việt Nam (USD/tấn)', fontsize=12)
axes[1].set_ylabel('Kim Ngạch Xuất Khẩu (Triệu USD)', fontsize=12)
axes[1].set_title('Kim Ngạch XK vs Giá Việt Nam', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
cbar2 = plt.colorbar(axes[1].collections[0], ax=axes[1])
cbar2.set_label('Năm', fontsize=10)

plt.tight_layout()
plt.show()

# Calculate correlation
corr_world = df_export[['export_value_million_usd', 'price_world_usd_per_ton']].corr().iloc[0, 1]
corr_vn = df_export[['export_value_million_usd', 'price_vn_usd_per_ton']].corr().iloc[0, 1]

print(f"\nCorrelation Kim Ngạch vs Giá Thế Giới: {corr_world:.3f}")
print(f"Correlation Kim Ngạch vs Giá Việt Nam: {corr_vn:.3f}")

## Part 3: Sản Lượng vs Kim Ngạch Xuất Khẩu

Phân tích mối quan hệ giữa sản lượng xuất khẩu và kim ngạch

In [ ]:
# Merge production and export data
merged_prod_export = pd.merge(df_production, df_export, on='year', how='inner')

print(f"Merged production + export data: {merged_prod_export.shape}")
print("\nPreview:")
print(merged_prod_export[['year', 'output_tons', 'export_tons', 'export_value_million_usd']].head())

In [ ]:
# Scatter Plot: Sản Lượng vs Kim Ngạch
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Sản lượng sản xuất vs Kim ngạch
axes[0].scatter(merged_prod_export['output_tons'], merged_prod_export['export_value_million_usd'], 
                s=100, alpha=0.6, c=merged_prod_export['year'], cmap='RdYlGn', edgecolors='black')
axes[0].set_xlabel('Sản Lượng Sản Xuất (tấn)', fontsize=12)
axes[0].set_ylabel('Kim Ngạch Xuất Khẩu (Triệu USD)', fontsize=12)
axes[0].set_title('Sản Lượng Sản Xuất vs Kim Ngạch XK', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].ticklabel_format(style='plain', axis='x')
cbar1 = plt.colorbar(axes[0].collections[0], ax=axes[0])
cbar1.set_label('Năm', fontsize=10)

# Sản lượng xuất khẩu vs Kim ngạch
axes[1].scatter(merged_prod_export['export_tons'], merged_prod_export['export_value_million_usd'], 
                s=100, alpha=0.6, c=merged_prod_export['year'], cmap='coolwarm', edgecolors='black')
axes[1].set_xlabel('Sản Lượng Xuất Khẩu (tấn)', fontsize=12)
axes[1].set_ylabel('Kim Ngạch Xuất Khẩu (Triệu USD)', fontsize=12)
axes[1].set_title('Sản Lượng XK vs Kim Ngạch XK', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].ticklabel_format(style='plain', axis='x')
cbar2 = plt.colorbar(axes[1].collections[0], ax=axes[1])
cbar2.set_label('Năm', fontsize=10)

plt.tight_layout()
plt.show()

# Calculate correlations
corr_output = merged_prod_export[['output_tons', 'export_value_million_usd']].corr().iloc[0, 1]
corr_export = merged_prod_export[['export_tons', 'export_value_million_usd']].corr().iloc[0, 1]

print(f"\nCorrelation Sản Lượng Sản Xuất vs Kim Ngạch: {corr_output:.3f}")
print(f"Correlation Sản Lượng Xuất Khẩu vs Kim Ngạch: {corr_export:.3f}")

## Part 4: Scatter Plots Chi Tiết - Weather vs Production

Phân tích chi tiết các mối quan hệ giữa thời tiết và sản xuất

In [ ]:
# Detailed Scatter Plots: Weather factors vs Production metrics
import numpy as np

# Ensure all data is float (fix Decimal issue)
weather_vars = ['temperature', 'humidity', 'rain']
prod_vars = ['area_thousand_ha', 'output_tons', 'export_tons']

for col in weather_vars + prod_vars:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

fig, axes = plt.subplots(3, 3, figsize=(18, 15))

for i, weather in enumerate(weather_vars):
    for j, prod in enumerate(prod_vars):
        ax = axes[i, j]
        
        # Filter out NaN values for both variables
        valid_mask = merged_df[[weather, prod]].notna().all(axis=1)
        x_data = merged_df.loc[valid_mask, weather].astype(float)
        y_data = merged_df.loc[valid_mask, prod].astype(float)
        year_data = merged_df.loc[valid_mask, 'year']
        
        # Scatter plot with trend line
        scatter = ax.scatter(x_data, y_data, 
                  s=80, alpha=0.6, c=year_data, 
                  cmap='viridis', edgecolors='black', linewidth=0.5)
        
        # Add trend line
        if len(x_data) > 1:
            z = np.polyfit(x_data, y_data, 1)
            p = np.poly1d(z)
            ax.plot(x_data, p(x_data), 
                   "r--", alpha=0.8, linewidth=2, label='Trend line')
        
        # Calculate correlation
        corr = x_data.corr(y_data) if len(x_data) > 1 else 0
        
        # Labels and title
        ax.set_xlabel(weather.replace('_', ' ').title(), fontsize=10)
        ax.set_ylabel(prod.replace('_', ' ').title(), fontsize=10)
        ax.set_title(f'{weather.title()} vs {prod.replace("_", " ").title()}\n(r={corr:.3f})', 
                    fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

print("Các scatter plot chi tiết đã được tạo!")

## Part 5: Correlation Matrix Tổng Hợp

Ma trận tương quan tổng hợp cho tất cả các biến quan trọng

In [ ]:
# Merge all data: production + weather + export
full_merged = pd.merge(merged_df, df_export, on='year', how='inner')

# Select key variables for comprehensive correlation
key_vars = [
    'area_thousand_ha', 'output_tons', 'export_tons',
    'temperature', 'humidity', 'rain',
    'export_value_million_usd', 'price_world_usd_per_ton', 'price_vn_usd_per_ton'
]

# Create correlation matrix
comprehensive_corr = full_merged[key_vars].corr()

# Visualize with enhanced heatmap
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(comprehensive_corr, dtype=bool))  # Mask upper triangle
sns.heatmap(comprehensive_corr, 
            mask=mask,
            annot=True, 
            fmt=".2f",
            cmap='RdBu_r',
            center=0,
            square=True,
            linewidths=1,
            cbar_kws={"shrink": 0.8, "label": "Correlation Coefficient"})

plt.title('Ma Trận Tương Quan Tổng Hợp\n(Weather, Production, Export & Price)', 
          fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()
plt.show()

print("Ma trận tương quan tổng hợp đã được tạo!")
print(f"\nKích thước dữ liệu: {full_merged.shape}")

## Part 6: Top Correlations Summary

Tóm tắt các mối tương quan mạnh nhất


In [ ]:
# Extract and rank correlations
corr_pairs = []
for i in range(len(comprehensive_corr.columns)):
    for j in range(i+1, len(comprehensive_corr.columns)):
        corr_pairs.append({
            'Variable 1': comprehensive_corr.columns[i],
            'Variable 2': comprehensive_corr.columns[j],
            'Correlation': comprehensive_corr.iloc[i, j]
        })

corr_df = pd.DataFrame(corr_pairs)
corr_df['Abs_Correlation'] = corr_df['Correlation'].abs()
corr_df = corr_df.sort_values('Abs_Correlation', ascending=False)

print("=" * 80)
print("TOP 10 TƯƠNG QUAN MẠNH NHẤT")
print("=" * 80)
print(corr_df.head(10)[['Variable 1', 'Variable 2', 'Correlation']].to_string(index=False))

print("\n" + "=" * 80)
print("TOP 10 TƯƠNG QUAN YẾU NHẤT")
print("=" * 80)
print(corr_df.tail(10)[['Variable 1', 'Variable 2', 'Correlation']].to_string(index=False))

# Visualize top correlations with better formatting
top_10 = corr_df.head(10).copy()

# Tạo tên ngắn gọn và dễ đọc
def shorten_name(name):
    name_map = {
        'area_thousand_ha': 'Diện tích',
        'output_tons': 'Sản lượng',
        'export_tons': 'XK (tấn)',
        'temperature': 'Nhiệt độ',
        'humidity': 'Độ ẩm',
        'rain': 'Lượng mưa',
        'export_value_million_usd': 'Kim ngạch XK',
        'price_world_usd_per_ton': 'Giá TG',
        'price_vn_usd_per_ton': 'Giá VN'
    }
    return name_map.get(name, name)

# Tạo labels đẹp hơn
top_10['Label'] = top_10.apply(
    lambda row: f"{shorten_name(row['Variable 1'])} ↔ {shorten_name(row['Variable 2'])}", 
    axis=1
)

plt.figure(figsize=(14, 8))
colors = ['green' if x > 0 else 'red' for x in top_10['Correlation']]
bars = plt.barh(range(len(top_10)), top_10['Correlation'], color=colors, alpha=0.7, edgecolor='black')

# Thêm giá trị correlation vào cuối mỗi thanh
for idx, (bar, corr) in enumerate(zip(bars, top_10['Correlation'])):
    plt.text(corr + 0.02 if corr > 0 else corr - 0.02, 
             idx, f'{corr:.3f}', 
             va='center', ha='left' if corr > 0 else 'right',
             fontweight='bold', fontsize=10)

plt.yticks(range(len(top_10)), top_10['Label'], fontsize=11)
plt.xlabel('Hệ số tương quan (Correlation Coefficient)', fontsize=12, fontweight='bold')
plt.title('Top 10 Mối Tương Quan Mạnh Nhất\n(Xanh: tương quan dương | Đỏ: tương quan âm)', 
          fontsize=14, fontweight='bold', pad=15)
plt.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
plt.grid(True, alpha=0.3, axis='x')
plt.xlim(-1, 1.15)
plt.tight_layout()
plt.show()

print("\nPhân tích hoàn tất!")

In [ ]:
# Close database connection
connection.close()
print("Database connection closed")